In [4]:
# Import
import torch
import numpy as np
from torch.utils.data import Dataset
from get_data_tensors import get_laws

# [PAD]  Padding token 512 tokens per seqences                          0
# [UNK]  Used when a word is unknown to Bert                          100
# [CLS]  Appears at the start of every sequence                       101
# [SEP]  Indicates a seperator - between and end of sequences token   102
# [MASK] Used when masking tokens, masked language modelling (MLM)    103

In [ ]:
class Change(name):
    
    se

In [ ]:
class Law(name):
    
    fname = '../Data_Laws/'
    law_names = np.loadtxt(fname + 'done_with.txt', dtype=str, encoding='utf-8')
    #np.random.shuffle(laws)
    
    def __init__(self,name):
        self.name = name
        self.changes = []
        
    def num_changes(self):
        return len(self.changes)

    def get_name(self):
        return self.name
    
    def get_laws(split=0.9):
        ten = []
        
        num_data_training = int(split*len(laws))
    
        for i in range(num_data):
            print(laws[i])
            ten.append(get_old_change_new(laws[i]))
    
        return ten

In [ ]:
def get_tensors(ocn):
    
    input_ids = torch.from_numpy(np.load(ocn))
    mask = torch.ones(input_ids.size())
    
    input_id_chunks = input_ids.split(510)
    mask_chunks = mask.split(510)
    
    chunksize = 512
    
    input_id_chunks = list(input_id_chunks) 
    mask_chunks = list(mask_chunks) 
    
    for i in range(len(input_id_chunks)):
        input_id_chunks[i] = torch.cat([
            torch.Tensor([101]),input_id_chunks[i],torch.Tensor([102])
        ])
        mask_chunks[i] = torch.cat([
            torch.Tensor([1]),mask_chunks[i],torch.Tensor([1])
        ])
        
        # get required padding length
        pad_len = chunksize - input_id_chunks[i].shape[0]
        
        # check if tensor length satisfies required chunk size
        if pad_len > 0:
            
            # if padding length is more than 0, we must add padding
            input_id_chunks[i] = torch.cat([
                input_id_chunks[i], torch.Tensor([0] * pad_len)
            ])
            mask_chunks[i] = torch.cat([
                mask_chunks[i], torch.Tensor([0] * pad_len)
            ])
            
    input_ids = torch.stack(input_id_chunks)
    attentions_mask = torch.stack(mask_chunks)
    
    input_dict = {
        'input_ids': input_ids.long(),
        'attention_mask': attentions_mask.int()
    }
    
    return input_dict

In [ ]:
def get_old_change_new(law):

    law = str(law)
    fname = '../Data_Laws/' + law + '/'
    changes = np.loadtxt(fname + 'changes.txt', dtype=str, encoding='utf-8')
    
    ten_law = []
    
    if changes.shape == ():
        change = str(changes)
        old = get_tensors(fname + change + '/old.npy')
        cha = get_tensors(fname + change + '/change.npy')
        new = get_tensors(fname + change + '/new.npy')
        ocn = (old,cha,new)
        ten_law.append(ocn)
        return ten_law
    
    for change in changes:
        change = str(change)
        
        if law == 'KWG' and change == 'Nr7_2020-12-29':
            continue
            
        old = get_tensors(fname + change + '/old.npy')
        cha = get_tensors(fname + change + '/change.npy')
        new = get_tensors(fname + change + '/new.npy')
        ocn = (old,cha,new)
        ten_law.append(ocn)
        
    return ten_law

In [ ]:
# get_laws()
# print('Done!')

In [9]:
def masking_task(data):
    data_length = len(data)
    
    out = []
    for law in data:
        for change in law:
            old, change, new = change
            out.append(old)
            out.append(change)
            out.append(new)
        
    return out

In [12]:
data = get_laws(0.05)

BetrVG
AufenthG
GKG
UrhG
KVAV
IfSG


In [13]:
#data = masking_task(data)
#data

In [14]:
len(data)

6

In [32]:
class LawDatasetForMasking(Dataset):
    
    def __init__(self, data):
        data = self.masking_task(data)
        self.data = data
        
    def __len__(self):
        return len(self.data)

    def masking_task(self, data):
        out = []
        for law in data:
            for change in law:
                old, change, new = change
                out.append(old)
                out.append(change)
                out.append(new)
        return out
    
    def __getitem__(self, idx):
        return self.data[idx]

In [33]:
train = LawDatasetForMasking(data)

In [34]:
train.__getitem__(500)

{'input_ids': tensor([[  101, 29977, 13981,  ...,   311,  3461,   102],
         [  101,   132,   195,  ...,  9837,  2820,   102],
         [  101, 30889,  2506,  ..., 22672, 12842,   102],
         ...,
         [  101,  5607,   197,  ...,  5465,   566,   102],
         [  101,  9837, 29519,  ..., 18191,   222,   102],
         [  101, 16081,  4884,  ...,     0,     0,     0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0]], dtype=torch.int32)}

In [31]:
# print(type(data)) # list: Gesetzten die genutzt werden
# print(len(data))  # int(split*len(laws))

# print(type(data[0])) # list: Changes die es gab pro Gesetz
# print(len(data[0]))  # Num an Changes
      
# print(type(data[0][0])) # tuple: old, change, new
# print(len(data[0][0]))  # 3

# print(type(data[0][0][0])) # dict: key: ('input_ids', 'attention_mask') values: there pt_tensor representation

# print(data[0][0][0]['input_ids'].shape) #shape = (__,512)
# print(data[0][0][0]['input_ids']) #pt_tensor long: attual data
# print(data[0][0][0]['attention_mask']) #pt_tensor int: only 1 (attention) or 0 (no attention)